<a href="https://colab.research.google.com/github/addo561/Pytorch-book-chapters/blob/main/chapter4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data preprocessing

In [1]:
#handle missing values
import pandas as pd
from io import StringIO
csv_data = \
 '''A,B,C,D
 1.0,2.0,3.0,4.0
 5.0,6.0,,8.0
 10.0,11.0,12.0,'''
df = pd.read_csv(StringIO(csv_data))
df


,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [2]:
df.isnull().sum()

,0
A,0
B,0
C,1
D,1


In [4]:
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [3]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [6]:
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [7]:
#use imputer to fill or use fillna
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


In [9]:
from sklearn.impute import SimpleImputer
import numpy as np
imr = SimpleImputer(missing_values=np.nan,strategy='mean')
imr= imr.fit(df.values)
data = imr.transform(df.values)
data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

handling Categorical data

In [10]:
df = pd.DataFrame([
    ['green','M',10,'class1'],
    ['red','L',11,'class2'],
    ['blue','XL',13,'class1'],
])

df.columns =  ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10,class1
1,red,L,11,class2
2,blue,XL,13,class1


In [11]:
map = {'XL':3,
       'L':2,
       'M':1}

In [13]:
df['size'] = df['size'].map(map)

In [14]:
df

,color,size,price,classlabel
0,green,1,10,class1
1,red,2,11,class2
2,blue,3,13,class1


In [15]:
inv = {v:k for k,v in map.items()}
df['size'].map(inv)

,size
0,M
1,L
2,XL


In [16]:
class_mapping = {label:idxs for idxs,label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [17]:
df['classlabel']= df['classlabel'].map(class_mapping)

In [18]:
df

,color,size,price,classlabel
0,green,1,10,0
1,red,2,11,1
2,blue,3,13,0


In [19]:
inv = {v:k for k,v in class_mapping.items()}
df['classlabel']= df['classlabel'].map(inv)

In [20]:
df

,color,size,price,classlabel
0,green,1,10,class1
1,red,2,11,class2
2,blue,3,13,class1


In [26]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le = LabelEncoder()
y = le.fit_transform(df.classlabel.values)

In [27]:
y

array([0, 1, 0])

In [28]:
le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

In [29]:
X = df[['color', 'size', 'price']].values

In [30]:
le = LabelEncoder()
X[:,0] = le.fit_transform(X[:,0])
X

array([[1, 1, 10],
       [2, 2, 11],
       [0, 3, 13]], dtype=object)

In [33]:
ohe = OneHotEncoder()
ohe.fit_transform(X[:,0].reshape(-1,1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [35]:
#col tranformer
from sklearn.compose import ColumnTransformer
c_tranf = ColumnTransformer([
    ('onehot',OneHotEncoder(),[0]),
    ('nothing','passthrough',[1,2])

])

c_tranf.fit_transform(X).astype(float)

array([[ 0.,  1.,  0.,  1., 10.],
       [ 0.,  0.,  1.,  2., 11.],
       [ 1.,  0.,  0.,  3., 13.]])

In [39]:
#use dummy
pd.get_dummies(df[['price', 'color', 'size']],drop_first=True)

,price,size,color_green,color_red
0,10,1,True,False
1,11,2,False,True
2,13,3,False,False


In [40]:
#col tranformer
from sklearn.compose import ColumnTransformer
c_tranf = ColumnTransformer([
    ('onehot',OneHotEncoder(categories='auto',drop='first'),[0]),
    ('nothing','passthrough',[1,2])

])

c_tranf.fit_transform(X).astype(float)

array([[ 1.,  0.,  1., 10.],
       [ 0.,  1.,  2., 11.],
       [ 0.,  0.,  3., 13.]])

In [44]:
df = pd.DataFrame([
    ['green','M',10,'class1'],
    ['red','L',11,'class2'],
    ['blue','XL',13,'class1'],
])

df.columns =  ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10,class1
1,red,L,11,class2
2,blue,XL,13,class1


In [45]:
df['x > M'] = df['size'].apply(lambda x :  1 if x in {'L','XL'} else 0)
df['x > L'] = df['size'].apply(lambda x :  1 if x =='XL' else 0)

In [46]:
df

,color,size,price,classlabel,x > M,x > L
0,green,M,10,class1,0,0
1,red,L,11,class2,1,0
2,blue,XL,13,class1,1,1


splitting datasets